In [1]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense

from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset = pd.read_csv('/content/drive/My Drive/semantic-question-matching/Features/final_features.csv', delimiter='\t')
dataset.head(2)

,question1,question2,is_duplicate,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,wmd,norm_wmd,cosine_distance,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,66,57,9,20,20,14,12,10,93,95,98,100,89,100,93,0.564615,0.217555,0.037908,3.774843,1.0,75.949318,0.275348,0.125323,0.137314,0.008893,-0.099771,0.108845,0.344742
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,51,88,-37,21,29,8,13,4,66,86,73,100,75,86,63,3.772346,1.368796,0.574596,15.130415,1.0,190.766894,1.072004,0.482108,0.648993,0.027151,0.060190,0.310524,0.033802


In [3]:
dataset.shape

(404290, 31)

In [4]:
dataset.columns

Index(['question1', 'question2', 'is_duplicate', 'len_q1', 'len_q2',
       'diff_len', 'len_char_q1', 'len_char_q2', 'len_word_q1', 'len_word_q2',
       'common_words', 'fuzz_qratio', 'fuzz_WRatio', 'fuzz_partial_ratio',
       'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio',
       'fuzz_token_set_ratio', 'fuzz_token_sort_ratio', 'wmd', 'norm_wmd',
       'cosine_distance', 'cityblock_distance', 'jaccard_distance',
       'canberra_distance', 'euclidean_distance', 'minkowski_distance',
       'braycurtis_distance', 'skew_q1vec', 'skew_q2vec', 'kur_q1vec',
       'kur_q2vec'],
      dtype='object')

In [0]:
dataset.replace(to_replace=np.inf, value=np.nan, inplace=True)
dataset.dropna(axis=0, inplace=True)

In [0]:
#diff_len, common_words,  fuzz_qratio ,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,  fuzz_partial_token_sort_ratio,  fuzz_token_set_ratio, fuzz_token_sort_ratio,cosine_distance, cityblock_distance, canberra_distance,euclidean_distance,minkowski_distance  and  braycurtis_distance. 
X = dataset.iloc[:, [5,10,11,12,13,14,15,16,17,18,20,21,23,24,25,26]].values
y = dataset.iloc[:, 2].values

In [7]:
np.any(np.isinf(X))

False

In [8]:
np.any(np.isnan(X))

False

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
print(np.sum(Y_train))
print(np.sum(Y_test))

117361
29409


In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [0]:
classifier=Sequential()
classifier.add(Dense(input_dim=16, kernel_initializer="uniform", units=8, activation="relu"))
classifier.add(Dense(kernel_initializer="uniform", units=8, activation="relu"))
classifier.add(Dense(kernel_initializer="uniform", units=8, activation="relu"))
classifier.add(Dense(kernel_initializer="uniform", units=8, activation="relu"))
classifier.add(Dense(kernel_initializer="uniform", units=8, activation="relu"))
classifier.add(Dense(kernel_initializer="uniform", units=8, activation="relu"))
classifier.add(Dense(kernel_initializer="uniform", units=1, activation="sigmoid"))

In [0]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
classifier.fit(X_train,Y_train,batch_size=10,epochs=20)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
314651/314651 [==============================] - 40s 126us/step - loss: 0.5214 - acc: 0.6934
Epoch 2/20
314651/314651 [==============================] - 39s 124us/step - loss: 0.5075 - acc: 0.7149
Epoch 3/20
314651/314651 [==============================] - 39s 124us/step - loss: 0.5015 - acc: 0.7188
Epoch 4/20
314651/314651 [==============================] - 39s 125us/step - loss: 0.4996 - acc: 0.7200
Epoch 5/20
314651/314651 [==============================] - 39s 124us/step - loss: 0.4990 - acc: 0.7196
Epoch 6/20
314651/314651 [==============================] - 39s 123us/step - loss: 0.4984 - acc: 0.7203
Epoch 7/20
314651/314651 [==============================] - 39s 123us/step - loss: 0.4978 - acc: 0.7206
Epoch 8/20
314651/314651 [==============================] - 40s 127us/step - loss: 0.4978 - acc: 0.7206
Epoch 9/20
314651/314651 [==============================] - 39s 123us/step - loss: 0.4974 - acc: 0.7211
Epoch 10/20
3146

In [0]:
y_pred = classifier.predict(X_test)
y_pred=(y_pred>0.5)

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
print(cm)

[[35960 13294]
 [ 8548 20861]]


In [0]:
import pickle
pickle.dump(classifier, open('ANN.model', 'wb'))